In [3]:
import pandas as pd
import statsmodels.api as sm
fil = pd.read_excel('/Users/macbook/Desktop/bond_finance_2.xlsx')

In [218]:
fil = fil[fil['listed_date']>='2011-01-01']

In [219]:
set_sql('mysql','210.34.5.184','group1','Group1.321','group1')
test1 = search_sql('select * from static_bond')

In [220]:
com = search_sql('select * from static_institution')

In [221]:
fil['institution_id'] = map(unicode,list(fil['institution_id']))

In [222]:
fik = pd.merge(fil,com,on='institution_id')

In [227]:
fik.shape

(17328, 122)

In [223]:
da1 = pd.DataFrame({'security_id':fik['security_id'],'date':fik['listed_date']})

In [224]:
da2 = pd.DataFrame({'security_id':fik['security_id'],'date':fik['delisted_date']})

In [225]:
add1 = pd.merge(pd.merge(da1,dates),pd.merge(da2,dates),on='security_id')

In [229]:
fik = pd.merge(add1,fik,on='security_id')

In [235]:
fik.columns

Index([u'date_x', u'security_id', u'index_x', u'isOpen_x', u'number1_x',
       u'week_x', u'year_x', u'month_x', u'yw_x', u'ym_x',
       ...
       u'outward_y', u'net_asset_y', u'guarantee_rate_y', u'revenue_y',
       u'guarantee_bond_y', u'level1_id_y', u'level2_id_y', u'rating_date_y',
       u'level_name_y', u'warning_state_y'],
      dtype='object', length=140)

In [248]:
len(fik[fik['ym_x']>='2013-09'])/14716

0.938842076651264

In [254]:
fik[fik['ym_x']>='2013-09']['default'].value_counts()


0    13761
1       55
Name: default, dtype: int64

In [259]:
fik[fik['ym_x']<='2012-11']['default'].value_counts()

0    395
1      6
Name: default, dtype: int64

In [55]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-


from __future__ import division, unicode_literals
import pandas as pd
import pymysql
import pymssql
from sqlalchemy import create_engine
import re
import numpy as np
import statsmodels.api as sm
from scipy.stats import *
from sklearn import preprocessing
import datetime
import tushare as ts
import re

gdb = ''
ghost = ''
guser = ''
gpassword = ''
gdatabase = ''
conn = ''
engine = ''



def set_sql(gdb, ghost, guser, gpassword, gdatabase):
    global db, host, user, password, database, engine, conn
    db = gdb
    host = ghost
    user = guser
    password = gpassword
    database = gdatabase
    engine = set_engine()
    conn = set_conn()


def set_engine():
    global engine
    try:
        engine.close()
    except:
        pass
    if db.lower() == 'mysql':
        engine = create_engine('mysql+pymysql://' + user + ':' + password + '@' + host + '/' + database)
    if db.lower() == 'mssql':
        engine = create_engine('mssql+pyodbc://' + user + ':' + password + '@' + host + '/' + database)
    return engine



def set_conn():
    if db.lower() == 'mysql':
        conn = pymysql.connect(host=host, user=user, password=password, db=database, charset='utf8')
    if db.lower() == 'mssql':
        conn = pymssql.connect(host=host, user=user, password=password, database=database, charset="utf8")
    return conn


def execute_sql(sql_str, conn):
    cursor = conn.cursor()
    cursor.execute(sql_str)
    conn.commit()



def search_sql(sql_str):
    s = pd.read_sql(sql_str, conn)
    return s



def to_sql(df, table_name):
    try:
        df.to_sql(table_name, engine, if_exists='append')
    except IOError as e:
        print('to sql error! \n ' + e)

In [58]:
set_sql('mysql', '172.16.8.184', 'qc_data', 'wisesoe.qc', 'qc_data')
data = search_sql('select * from induindextest')

In [59]:
data['date'] = map(unicode,list(data['date']))

In [60]:
dates = search_sql('select * from dates')
macro = search_sql('select * from macro')
macro = pd.merge(macro,dates,on='date')
epu = search_sql('select * from EPU')[1:]
epu.columns = ['year','month','epu']
epu['epu'] = map(float,list(epu['epu']))
ym = []
for i in range(len(epu)):
    year = int(epu.iloc[i]['year'])
    month = int(epu.iloc[i]['month'])
    if month<10:
        ym.append(str(year)+'0'+str(month))
    else:
        ym.append(str(year)+str(month))
epu['ym'] = ym
del epu['year']
del epu['month']
index = search_sql('select ret,date from zhishu where qtid="000300.SH" and date>="2011-01-01"')
index['date'] = map(str,list(index['date']))
index = pd.merge(index,dates,on='date')

In [61]:
data = pd.merge(data,dates,on='date')

In [84]:
data = pd.merge(data,epu)
fin =pd.merge(data,index,on='date')

In [86]:
betas = []
for i in fin.groupby('indu').groups.keys():
    hangye = fin.groupby('indu').get_group(i).groupby('ym_y')['ret_x'].sum().values[1:]
    zhishu = fin.groupby('indu').get_group(i).groupby('ym_y')['ret_y'].sum().values[1:]
    bodong = fin.groupby('indu').get_group(i).groupby('ym_y')['ret_y'].std().values
    bodong = pd.Series(bodong).fillna(pd.Series(bodong).mean())[:-1]
    epuu = fin.groupby('indu').get_group(i).groupby('ym_y')['epu'].mean().values[:-1]
    yujing = macro[240:-4]['yujing']
    yizhi = macro[240:-4]['yizhi']
    #smb = finn.groupby('indu').get_group(i).groupby('ym_x')['SMB'].sum().values
    #hml = finn.groupby('indu').get_group(i).groupby('ym_x')['HML'].sum().values
    xs = np.column_stack([zhishu,np.log(bodong),np.log(epuu),np.log(yujing),np.log(yizhi)])
    kong = [i]
    model = sm.OLS(hangye,sm.add_constant(xs)).fit(method='qr')
    for i in range(len(model.params)):
        kong.append(model.params[i])
    kong.append(abs(bodong.std()*model.params[2]))
    kong.append(abs(epuu.std()*model.params[3]))
    kong.append(abs(yujing.std()*model.params[4]))
    kong.append(abs(yizhi.std()*model.params[5]))
    betas.append(kong)
    


In [87]:
pd.DataFrame(betas)

,0,1,2,3,4,5,6,7,8,9,10
0,42,2.844108,1.164800,0.035624,-0.015950,0.087450,-0.650783,0.000247,2.188177,1.182075,1.978757
1,50,1.110832,1.065250,0.029596,-0.014096,-0.038421,-0.158686,0.000205,1.933846,0.519348,0.482497
2,60,7.980346,0.983145,0.046659,-0.031166,0.285767,-1.936052,0.000323,4.275640,3.862753,5.886722
3,61,8.015896,1.059407,0.062690,-0.027188,0.282348,-1.929625,0.000434,3.729995,3.816537,5.867179
4,62,5.320435,1.071581,0.052957,-0.041754,0.123371,-1.177994,0.000367,5.728301,1.667630,3.581785
5,63,4.454954,1.005055,0.062646,-0.028747,0.158509,-1.028500,0.000434,3.943810,2.142594,3.127237
6,24,4.641354,1.066442,0.029143,-0.024361,0.111083,-1.063284,0.000202,3.342090,1.501530,3.233000
7,25,5.107646,1.055386,0.048222,-0.026306,0.129194,-1.161662,0.000334,3.608959,1.746333,3.532126
8,26,5.413064,1.056854,0.031779,-0.034658,0.100622,-1.207499,0.000220,4.754728,1.360126,3.671497
9,27,5.809781,1.057102,0.044664,-0.023414,0.156313,-1.348866,0.000309,3.212132,2.112907,4.101336


In [242]:
fin['ym_x'].unique()[2:33]

array([u'201103', u'201104', u'201105', u'201106', u'201107', u'201108',
       u'201109', u'201110', u'201111', u'201112', u'201201', u'201202',
       u'201203', u'201204', u'201205', u'201206', u'201207', u'201208',
       u'201209', u'201210', u'201211', u'201212', u'201301', u'201302',
       u'201303', u'201304', u'201305', u'201306', u'201307', u'201308',
       u'201309'], dtype=object)

In [276]:
betas = []
for i in fin.groupby('indu').groups.keys():
    for j in range(21,80):
        hangye = list(fin.groupby('indu').get_group(i).groupby('ym_x')['ret_x'].sum().values)[j-19:j+1]
        zhishu = fin.groupby('indu').get_group(i).groupby('ym_x')['ret_y'].sum().values[j-19:j+1]
        bodong = fin.groupby('indu').get_group(i).groupby('ym_x')['ret_y'].std().values[j-20:j]
        bodong = list(pd.Series(bodong).fillna(pd.Series(bodong).mean()))
        epuu = fin.groupby('indu').get_group(i).groupby('ym_x')['epu'].mean().values[j-20:j]
        yj = list(yujing)[j-19:j+1]
        yz = list(yizhi)[j-19:j+1]
        #smb = finn.groupby('indu').get_group(i).groupby('ym_x')['SMB'].sum().values
        #hml = finn.groupby('indu').get_group(i).groupby('ym_x')['HML'].sum().values
        xs = np.column_stack([zhishu,bodong,np.log(epuu),yj,yz])
        kong = [i,fin['ym_x'].unique()[j+1]]
        model = sm.OLS(hangye,sm.add_constant(xs)).fit(method='qr')
        kong.append(model.params[0])
        kong.append(model.params[1])
        kong.append(np.array(hangye[:-3]).sum()-np.array(zhishu[:-3]).sum())
        kong.append(np.array(hangye[:-5]).sum()-np.array(zhishu[:-5]).sum())
        kong.append(np.array(hangye[:-7]).sum()-np.array(zhishu[:-7]).sum())
        kong.append(abs(pd.Series(bodong).std()*model.params[2]))
        kong.append(abs(epuu.std()*model.params[3]))
        kong.append(abs(np.array(yj).std()*model.params[4]))
        kong.append(abs(np.array(yz).std()*model.params[5]))
        betas.append(kong)

In [277]:
para = pd.DataFrame(betas)

In [333]:
para = pd.DataFrame(betas)
para[0] = para[0].map(hy)
para.columns=['code','ym','alpha','beta','mom1','mom2','mom3','ecbd2','ecep2','ecyj2','ecyz2']

In [334]:
adds = []
for i in para.groupby('code').groups.keys():
    ki= para.groupby('code').get_group(i).groupby('ym').mean()
    ki['industry_name'] = np.repeat(i,len(ki))
    adds.append(ki)

In [335]:
adds = pd.concat(adds)

In [336]:
adds['ym'] = adds.index

In [337]:
months = fin['ym_x'].unique()

In [338]:
add2 = []
for i in range(len(fik)):
    every = fik.iloc[i]
    start,end = every['ym_x'],every['ym_y']
    indu = every['industry_name_x']
    ek = adds[adds['industry_name']==indu]
    add2.append(ek[(ek['ym']>=start) & (ek['ym']<=end)].mean())



In [339]:
add2 = pd.DataFrame(add2)
del add2['industry_name']
del add2['ym']

In [341]:
fik2 = pd.concat([fik,add2],axis=1)

In [305]:
fin['ym_x'].unique()>='201111'

array([False, False, False, False, False, False, False, False, False,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True], dtype=bool)

In [265]:
fik['ym_x'].iloc[0]

u'201111'

In [267]:
fik.columns

Index([u'date_x', u'security_id', u'index_x', u'isOpen_x', u'number1_x',
       u'week_x', u'year_x', u'month_x', u'yw_x', u'ym_x',
       ...
       u'outward_y', u'net_asset_y', u'guarantee_rate_y', u'revenue_y',
       u'guarantee_bond_y', u'level1_id_y', u'level2_id_y', u'rating_date_y',
       u'level_name_y', u'warning_state_y'],
      dtype='object', length=140)

In [94]:
for i in fik['industry_name'].unique():
    print i

制造业
电力、热力、燃气及水生产和供应业
交通运输、仓储和邮政业
金融业
建筑业
水利、环境和公共设施管理业
采矿业
综合
房地产业
信息传输、软件和信息技术服务业
批发和零售业
居民服务、修理和其他服务业
卫生和社会工作

租赁和商务服务业
文化、体育和娱乐业
住宿和餐饮业
科学研究和技术服务业
农、林、牧、渔业


In [105]:
print fik.groupby('industry_name')['institution_id'].count()

industry_name
                       52
交通运输、仓储和邮政业           650
住宿和餐饮业                 15
信息传输、软件和信息技术服务业        27
农、林、牧、渔业               23
制造业                  1608
卫生和社会工作                11
居民服务、修理和其他服务业          41
建筑业                   558
房地产业                  361
批发和零售业                504
文化、体育和娱乐业              20
水利、环境和公共设施管理业          69
电力、热力、燃气及水生产和供应业      634
科学研究和技术服务业              8
租赁和商务服务业               48
综合                    707
采矿业                   416
金融业                 13378
Name: institution_id, dtype: int64


In [142]:
print fik.groupby('industry_name')['default'].sum()

industry_name
                     0
交通运输、仓储和邮政业          3
住宿和餐饮业               1
信息传输、软件和信息技术服务业      0
农、林、牧、渔业             0
制造业                 68
卫生和社会工作              0
居民服务、修理和其他服务业        0
建筑业                 11
房地产业                 0
批发和零售业               9
文化、体育和娱乐业            0
水利、环境和公共设施管理业        0
电力、热力、燃气及水生产和供应业     2
科学研究和技术服务业           1
租赁和商务服务业             0
综合                   7
采矿业                  7
金融业                  0
Name: default, dtype: int64


In [129]:
hy ={'33':'制造业','34':'制造业','30':'制造业','25':'制造业','26':'制造业','60':'制造业','28':'制造业','27':'制造业',
     '22':'制造业','35':'卫生和社会工作','23':'建筑业','24':'建筑业','42':'房地产业','31':'批发和零售业','36':'批发和零售业',
    '63':'文化、体育和娱乐业','20':'电力、热力、燃气及水生产和供应业','40':'金融业','41':'金融业','11':'采矿业','12':'采矿业',
    '70':'综合','32':'住宿和餐饮业','50':'交通运输、仓储和邮政业','60':'信息传输、软件和信息技术服务业',
    '61':'信息传输、软件和信息技术服务业','62':'信息传输、软件和信息技术服务业','37':'农、林、牧、渔业','10':'采矿业','21':'采矿业'}


In [130]:
beta = pd.DataFrame(betas)
beta[0] = beta[0].map(hy)
beta.columns=['code','con','zhishu','bodong','epuu','yujing','yizhi','ecbd','ecep','ecyj','ecyz']

In [154]:
kk1 = pd.DataFrame(beta.groupby('code')['ecbd'].mean()).to_dict()['ecbd']
kk2 = pd.DataFrame(beta.groupby('code')['ecep'].mean()).to_dict()['ecep']
kk3 = pd.DataFrame(beta.groupby('code')['ecyj'].mean()).to_dict()['ecyj']
kk4 = pd.DataFrame(beta.groupby('code')['ecyz'].mean()).to_dict()['ecyz']

ValueError: If using all scalar values, you must pass an index

In [155]:
fik['ecep'] = fik['industry_name'].map(kk2)
fik['ecbd'] = fik['industry_name'].map(kk1)
fik['ecyj'] = fik['industry_name'].map(kk3)
fik['ecyz'] = fik['industry_name'].map(kk4)

In [156]:
fil = fik.iloc[fik['ecep'].dropna().index]

In [152]:
beta.groupby('code')['ecep'].mean()

code
交通运输、仓储和邮政业         1.933846
住宿和餐饮业              3.617383
信息传输、软件和信息技术服务业     4.577979
农、林、牧、渔业            2.652808
制造业                 3.814441
卫生和社会工作             3.780977
建筑业                 2.984963
房地产业                2.188177
批发和零售业              2.419593
文化、体育和娱乐业           3.943810
电力、热力、燃气及水生产和供应业    1.618569
综合                  2.932432
采矿业                 2.038712
金融业                 0.754702
Name: ecep, dtype: float64

In [167]:
fil.shape

(18912, 75)

In [183]:
import statsmodels

In [178]:
xs =pd.DataFrame([fil['default'],fil['ROE'],fil['ecep'],fil['ecyj'],fil['ecyz'],fil['ecbd']]).T

In [179]:
xs = xs.dropna()

In [182]:
sm.OLS(xs['default'],sm.add_constant(np.column_stack([xs['ROE'],xs['ecyj'],xs['ecbd'],xs['ecyz']]))).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                default   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     198.1
Date:                Tue, 20 Feb 2018   Prob (F-statistic):          1.01e-166
Time:                        23:42:03   Log-Likelihood:                 22955.
No. Observations:               18837   AIC:                        -4.590e+04
Df Residuals:                   18832   BIC:                        -4.586e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0197      0.002      8.780      0.000       0.015       0.024
x1            -0.0027      0.000    -20.805      0.000      -0.003      -0.002
x2            -0.0110      0.004     -2.974      0.003      -0.018      -0.004
x3           -44.3939      8.457     -5.249      0.000     -60.971     -27.817
x4             0.0136      0.002      5.747      0.000       0.009       0.018
==============================================================================
Omnibus:                    33181.018   Durbin-Watson:                   1.260
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         24013526.836
Skew:                          12.934   Prob(JB):                         0.00
Kurtosis:                     175.992   Cond. No.                     1.84e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.84e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [188]:
model = statsmodels.discrete.discrete_model.Logit(xs['default'],sm.add_constant(np.column_stack([xs['ROE'],xs['ecyj'],xs['ecbd'],xs['ecyz']])))

In [193]:
pre = model.predict(model.fit().params,sm.add_constant(np.column_stack([xs['ROE'],xs['ecyj'],xs['ecbd'],xs['ecyz']])))

         Current function value: 0.052876
         Iterations: 35


/Users/macbook/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [207]:
(xs['default']-map(float,pre>0.5)).value_counts()


 0.0    18713
 1.0       88
-1.0       36
Name: default, dtype: int64

In [366]:
xs = pd.DataFrame([fik2['default'],fik2['ROE'],fik2['ecep2'],fik2['ecyj2'],fik2['ecyz2'],fik2['ecbd2'],fik2['mom2']]).T

In [367]:
xs = xs.dropna()

In [368]:
xs.corr()

,default,ROE,ecep2,ecyj2,ecyz2,ecbd2,mom2
default,1.000000,-0.171288,-0.027517,0.122003,0.084483,0.122078,0.096224
ROE,-0.171288,1.000000,0.089711,-0.475963,-0.250472,-0.423466,-0.262483
ecep2,-0.027517,0.089711,1.000000,-0.146960,0.031969,0.026180,0.136374
ecyj2,0.122003,-0.475963,-0.146960,1.000000,0.708708,0.635966,0.586428
ecyz2,0.084483,-0.250472,0.031969,0.708708,1.000000,0.779656,0.614912
ecbd2,0.122078,-0.423466,0.026180,0.635966,0.779656,1.000000,0.480532
mom2,0.096224,-0.262483,0.136374,0.586428,0.614912,0.480532,1.000000


In [369]:
y = xs['default']

In [370]:
del xs['default']

In [373]:
sm.OLS(y,xs).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                default   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     81.79
Date:                Wed, 21 Feb 2018   Prob (F-statistic):          5.83e-101
Time:                        01:06:30   Log-Likelihood:                 16873.
No. Observations:               13600   AIC:                        -3.373e+04
Df Residuals:                   13594   BIC:                        -3.369e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
ROE           -0.0011      0.000    -10.418      0.000      -0.001      -0.001
ecep2          0.0013      0.001      2.387      0.017       0.000       0.002
ecyj2          0.3887      0.072      5.386      0.000       0.247       0.530
ecyz2         -0.2933      0.074     -3.961      0.000      -0.439      -0.148
ecbd2          0.3528      0.065      5.397      0.000       0.225       0.481
mom2           0.0319      0.006      5.329      0.000       0.020       0.044
==============================================================================
Omnibus:                    24319.439   Durbin-Watson:                   1.878
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         19027274.239
Skew:                          13.306   Prob(JB):                         0.00
Kurtosis:                     184.299   Cond. No.                     1.78e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.78e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [377]:
model = statsmodels.discrete.discrete_model.Logit(y,xs)

In [380]:
model.fit().summary()

Optimization terminated successfully.
         Current function value: 0.033299
         Iterations 11


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                default   No. Observations:                13600
Model:                          Logit   Df Residuals:                    13594
Method:                           MLE   Df Model:                            5
Date:                Wed, 21 Feb 2018   Pseudo R-squ.:                -0.04492
Time:                        01:08:05   Log-Likelihood:                -452.87
converged:                       True   LL-Null:                       -433.40
                                        LLR p-value:                     1.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ROE           -0.1419      0.010    -14.069      0.000      -0.162      -0.122
ecep2         -2.0074      0.127    -15.814      0.000      -2.256      -1.759
ecyj2        -14.1347     11.116     -1.272      0.204     -35.922       7.652
ecyz2        -28.6628     15.543     -1.844      0.065     -59.126       1.800
ecbd2         24.9993      9.775      2.557      0.011       5.840      44.159
mom2           0.1610      0.848      0.190      0.849      -1.501       1.823
==============================================================================

Possibly complete quasi-separation: A fraction 0.12 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

In [381]:
pre = model.predict(model.fit().params,xs)

Optimization terminated successfully.
         Current function value: 0.033299
         Iterations 11


In [395]:
(y-map(float,pre>0.3)).value_counts()

 0.0    13522
 1.0       61
-1.0       17
Name: default, dtype: int64

In [390]:
pre[pre>0.5]

array([ 0.96266874,  0.95622091,  0.92642125,  0.55691373,  0.91683613,
        0.77718689,  0.9500811 ,  0.65426162,  0.87913292,  0.69752659,
        0.52528697,  0.63731458,  0.64522404,  0.99993339,  0.99993249,
        0.62995302])

In [396]:
y.value_counts()

0.0    13531
1.0       69
Name: default, dtype: int64